In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import cv2 as cv
import scipy.io as sio

%matplotlib widget

import sys  
sys.path.insert(0, 'lib')
from iplabs import IPLabViewer as viewer

# Print colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [48]:
# Load the entire sim dataset into a numpy array
def load_dataset(path_to_raw):
    ls = []
    path_to_raw = Path(path_to_raw)

    # Read images and resize them
    print('Collecting images...   0%', end='\r')
    subdir_list = [f for f in path_to_raw.glob('*')]
    for i, f in enumerate(subdir_list):
        img = cv.imread(str(f), cv.IMREAD_GRAYSCALE)
        sz = min(img.shape)//2
        img = cv.resize(img[img.shape[0]//2 - sz:img.shape[0]//2 + sz, img.shape[1]//2 - sz:img.shape[1]//2 + sz], (1024, 1024))
        img = img / np.max(img)
        ls.append(img)
        print(f'Collecting images... {i/len(subdir_list)*100:3.0f}%', end='\r')
    
    print(f'Collecting images... {bcolors.OKGREEN}100%{bcolors.ENDC}')
        
    return np.array(ls)

In [49]:
dataset = load_dataset('DNN4SIM_data/DIV2K')

# Save dataset as .mat file

In [58]:
for i in range(4):
    with open(f'DNN4SIM_data/DIV2K_{i}.mat', 'wb') as f:
        sio.savemat(f, {'data' : dataset[i*dataset.shape[0]//4:(i+1)*dataset.shape[0]//4]})

# Generating the rosette

In [22]:
rosette_512 = sio.loadmat(f'DNN4SIM_data/rosette_512.mat')['rosette']
rosette_1024 = sio.loadmat(f'DNN4SIM_data/rosette_1024.mat')['rosette']
rosette_512 = cv.morphologyEx(rosette_512, cv.MORPH_CLOSE, np.ones((3,3)))
rosette_1024 = cv.morphologyEx(rosette_1024, cv.MORPH_CLOSE, np.ones((7,7)))

In [24]:
plt.close('all')
view = viewer([rosette_512, rosette_1024], subplots=(1,2))

Button(description='Show Widgets', style=ButtonStyle())

In [25]:
with open('DNN4SIM_data/rosette_512.npy', 'wb') as f:
    np.save(f, rosette_512)
with open('DNN4SIM_data/rosette_1024.npy', 'wb') as f:
    np.save(f, rosette_1024)

# Generate NumPy dataset from reconstruction .mat files

In [30]:
def apply_rosette(img, rosette):
    FT = np.fft.fftshift(np.fft.fft2(img))
    FT[rosette == 0] = 1e-5
    return np.real(np.fft.ifft2(np.fft.ifftshift(FT)))

def apply_fft(img):
    FT = np.abs(np.fft.fftshift(np.fft.fft2(img)))
    FT[FT == 0] = 1e-5
    return 10*np.log10(FT)

In [7]:
#ds_recons = []
#ds = []
ds_wf = []
for i in range(4):
    #ds_recons.append(sio.loadmat(f'DNN4SIM_data/DIV2K_recons_{i}.mat')['recons_data'])
    #ds.append(sio.loadmat(f'DNN4SIM_data/DIV2K_{i}.mat')['data'])
    ds_wf.append(sio.loadmat(f'DNN4SIM_data/DIV2K_wf_{i}.mat')['wf_data'])

#ds_recons = np.concatenate(ds_recons, axis=0)
#ds = np.concatenate(ds, axis=0)
ds_wf = np.concatenate(ds_wf, axis=0)

In [7]:
for i, x in enumerate(ds_recons):
    ds_recons[i] = (x - np.min(x)) / (np.max(x) - np.min(x))

In [28]:
rosette_1024 = np.load('DNN4SIM_data/rosette_1024.npy')
for i, x in enumerate(ds):
    x = apply_rosette(x, rosette_1024)
    ds[i] = (x - np.min(x)) / (np.max(x) - np.min(x))

In [32]:
view = viewer([ds[234], apply_fft(ds[234])], subplots=(2,1))

Button(description='Show Widgets', style=ButtonStyle())

In [8]:
#with open('features.npy', 'wb') as f:
#    np.save(f, ds_recons)
#with open('labels.npy', 'wb') as f:
#    np.save(f, ds)
with open('wf.npy', 'wb') as f:
    np.save(f, ds_wf)